### Assignment 2: Fetching data from a Web API, store in database, schedule to run in background

#### Author: Melody Shi
#### Written on: Sep 30, 2018

In [1]:
import requests
from sqlalchemy import create_engine
from datetime import datetime, timedelta
import pytz

### Part 1: Identify a web API that returns data
The API used in this assignment is an Air Quality API provided by BeezoMeter(https://breezometer.com/air-quality-api/)

It is capable of:
- requesting historical Air Quality Index(AQI) data up to 14 days;
- requesting real-time Air Quality Index(AQI) data.

For the purpose of this assignment,
- @main_history_aqi() the main function of requesting historical data is called only once to fetch historical data and store in database;
- @main_real_time_aqi() the main function requesting real-time data is scheduled to run in the background on an hourly basis to fetch real-time data and store in database.

In [2]:
def get_history_time(hour):
    """
        Get past time by subtracting hours from now; 
        build a datetime string in a format that BeezoMeter API supports(e.g. 2018-09-20T18:00:00)
        
        Parameters
        ----------
        hour: int
            the number of hours to be subtracted from now
               
        Returns
        -------
            a formatted datetime string
    """
    timezone = pytz.timezone("US/Eastern")
    last_hour_date_time = datetime.now(timezone) - timedelta(hours = hour)
    return last_hour_date_time.strftime('%Y-%m-%dT%H:%M:%S')

In [3]:
def history_aqi(datetime):
    """
        Send request to BeezoMeter API and get back response of past-periods data
        
        Parameters
        ----------
        datetime: str
            a formatted datetime string 
               
        Returns
        -------
        resp: dict
            a json dictionary containing past-periods data
    """
    endpoint = "https://api.breezometer.com/baqi/?"
    history_params = {
    'datetime':datetime,
    'lat' : '40.7301249',
    'lon' : '-73.99429',
    'fields':"breezometer_aqi,random_recommendations,"
        +"dominant_pollutant_canonical_name,dominant_pollutant_description,"
        +"datetime,pollutants,dominant_pollutant_text",
    'key' : "8d9df1f14a9c4abb8f2e233d392bb7c8"  
    }
    resp = requests.get(endpoint,params = history_params).json()
    return resp

In [4]:
history_aqi('2018-09-20T18:00:00')

{'breezometer_aqi': 57,
 'datetime': '2018-09-20T18:00:00',
 'dominant_pollutant_canonical_name': 'pm10',
 'dominant_pollutant_description': 'Inhalable particulate matter (<10µm)',
 'dominant_pollutant_text': {'causes': 'Main sources are natural dust, smoke and pollen.',
  'effects': 'Particles enter the lungs and cause local and systemic inflammation in the respiratory system & heart, thus cause cardiovascular and respiratory diseases such as asthma and bronchitis.',
  'main': 'The dominant pollutant is inhalable particulate matter (PM10).'},
 'random_recommendations': {'children': 'You can go outside with the kids, but monitor their breathing & pay attention to notifications',
  'health': 'There is no real danger for people with health sensitivities. Just keep an eye out for changes in air quality for the next few hours',
  'inside': "You don't have to stay inside, but it's recommended that you keep tracking the air quality",
  'outside': 'If you wish to stay outside for a long time,

In [5]:
def real_time_aqi():
    """
        Send request to BeezoMeter API and get back response of real-time data
        
        Returns
        -------
        resp: dict
            a json dictionary containing real-time data
    """
    endpoint = "https://api.breezometer.com/baqi/?"
    real_time_params = {
    'lat' : '40.7301249',
    'lon' : '-73.99429',
    'fields':"breezometer_aqi,random_recommendations,"
        +"dominant_pollutant_canonical_name,dominant_pollutant_description,"
        +"datetime,pollutants,dominant_pollutant_text",
    'key' : "8d9df1f14a9c4abb8f2e233d392bb7c8"  
    }
    resp = requests.get(endpoint,params = real_time_params).json()
    return resp

In [6]:
real_time_aqi()

{'breezometer_aqi': 54,
 'datetime': '2018-10-02T23:00:00',
 'dominant_pollutant_canonical_name': 'pm10',
 'dominant_pollutant_description': 'Inhalable particulate matter (<10µm)',
 'dominant_pollutant_text': {'causes': 'Main sources are natural dust, smoke and pollen.',
  'effects': 'Particles enter the lungs and cause local and systemic inflammation in the respiratory system & heart, thus cause cardiovascular and respiratory diseases such as asthma and bronchitis.',
  'main': 'The dominant pollutant is inhalable particulate matter (PM10).'},
 'random_recommendations': {'children': 'You can go outside with the kids, but monitor their breathing & pay attention to notifications',
  'health': 'Exposure to air hazards is dangerous for people with health sensitivities, so it is important to monitor air quality at this time',
  'inside': "We're not going to tell you not to go outside, but you should continue tracking the air quality around you",
  'outside': 'Try to minimize long stays outd

### Part 2: Create a database and the corresponding set of tables for storing the data

In [7]:
def connect_db():
    """
        Connect to a database
        
        Returns
        -------
            an engine that supports SQL queries
    """
    conn_string = 'mysql://{user}:{password}@{host}/'.format(
        host = '35.196.156.52', 
        user = 'root',
        password = 'Lib8fGz3MPIiwFff')

    engine = create_engine(conn_string)
    con = engine.connect()
    return engine

In [8]:
def execute_create_table(engine):
    """
        Create a table in a database
        
        Parameters
        ----------
        engine:
            an engine that supports SQL queries
    """
    db_name = "aqi"
    table_name = 'NY_aqi'
    create_db_query = "CREATE DATABASE IF NOT EXISTS {db} DEFAULT CHARACTER SET 'utf8'".format(db=db_name)
    engine.execute(create_db_query)
    create_table_query = '''CREATE TABLE IF NOT EXISTS {db}.{table} 
                                    (breezometer_aqi int, 
                                    datetime varchar(250), 
                                    dominant_pollutant_canonical_name varchar(250),
                                    dominant_pollutant_description varchar(250),
                                    causes varchar(250),
                                    effects varchar(250),
                                    main varchar(250),
                                    recommendation_children varchar(250),
                                    recommendation_health varchar(250),
                                    recommendation_inside varchar(250),
                                    recommendation_outside varchar(250),
                                    recommendation_sport varchar(250),
                                    PRIMARY KEY(datetime)
                                    )'''.format(db=db_name, table=table_name)
    engine.execute(create_table_query)

In [9]:
def execute_insert_query(engine,data):
    """
        Insert entries into a table
        
        Parameters
        ----------
        engine:
            an engine that supports SQL queries
        data: dict
            response from API containing data           
    """
    db_name = 'aqi'
    table_name = 'NY_aqi'
    query_template = '''INSERT IGNORE INTO {db}.{table}(breezometer_aqi, 
                                            datetime,
                                            dominant_pollutant_canonical_name, 
                                            dominant_pollutant_description, 
                                            causes,
                                            effects,
                                            main,
                                            recommendation_children,
                                            recommendation_health,
                                            recommendation_inside,
                                            recommendation_outside,
                                            recommendation_sport) 
                        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'''.format(db=db_name, table=table_name)

    breezometer_aqi = int(data['breezometer_aqi'])
    datetime = data['datetime']
    dominant_pollutant_canonical_name = data['dominant_pollutant_canonical_name']
    dominant_pollutant_description = data['dominant_pollutant_description']
    causes = data['dominant_pollutant_text']['causes']
    effects = data['dominant_pollutant_text']['effects']
    main = data['dominant_pollutant_text']['main']
    recommendation_children = data['random_recommendations']['children']
    recommendation_health = data['random_recommendations']['health']
    recommendation_inside = data['random_recommendations']['inside']
    recommendation_outside = data['random_recommendations']['outside']
    recommendation_sport = data['random_recommendations']['sport']
    print("Inserting aqi at",datetime)
    query_parameters = (breezometer_aqi, datetime, dominant_pollutant_canonical_name, dominant_pollutant_description, 
                        causes, effects, main, recommendation_children, recommendation_health, recommendation_inside,
                       recommendation_outside, recommendation_sport)
    engine.execute(query_template, query_parameters)

### Part 3: Write Python code that connects to the API, fetches the data, and stores the data in the database

In [10]:
def main_history_aqi():
    """
        Fetch past-periods data up to 13 days from BeezoMeter API, store in database
    """
    history_time = [get_history_time(i) for i in range(1,313)] # create a list of well-formatted datetime strings
    engine = connect_db()
    execute_create_table(engine)
    for date in history_time:
        try:
            data = history_aqi(date) # fetch data
            execute_insert_query(engine, data) # store in database
        except:
            print("Error: unable to get historical data for "+date)

In [11]:
def main_real_time_aqi():
    """
        Fetch real-time data from BeezoMeter API, store in database
    """
    engine = connect_db()
    execute_create_table(engine)
    try:
        data = real_time_aqi() # fetch data
        execute_insert_query(engine, data) # store in database
    except:
        print("Error: unable to get real-time data.")

In [14]:
#only call once to get past-periods data, should not run as part of crontab job
#please see screenshots of database for functionality

main_history_aqi()

/usr/local/lib/python3.6/dist-packages/sqlalchemy/engine/default.py:509: Warning: (1007, "Can't create database 'aqi'; database exists")
  cursor.execute(statement, parameters)
/usr/local/lib/python3.6/dist-packages/sqlalchemy/engine/default.py:509: Warning: (1050, "Table 'NY_aqi' already exists")
  cursor.execute(statement, parameters)


Inserting aqi at 2018-10-02T20:00:00
Inserting aqi at 2018-10-02T19:00:00
Error: unable to get historical data for 2018-10-02T19:50:05
Inserting aqi at 2018-10-02T18:00:00
Inserting aqi at 2018-10-02T17:00:00
Error: unable to get historical data for 2018-10-02T17:50:05
Inserting aqi at 2018-10-02T16:00:00
Inserting aqi at 2018-10-02T15:00:00
Inserting aqi at 2018-10-02T14:00:00
Inserting aqi at 2018-10-02T13:00:00
Error: unable to get historical data for 2018-10-02T13:50:05
Inserting aqi at 2018-10-02T11:00:00
Error: unable to get historical data for 2018-10-02T12:50:05
Inserting aqi at 2018-10-02T11:00:00
Inserting aqi at 2018-10-02T10:00:00
Inserting aqi at 2018-10-02T09:00:00
Error: unable to get historical data for 2018-10-02T09:50:05
Inserting aqi at 2018-10-02T08:00:00
Error: unable to get historical data for 2018-10-02T08:50:05
Inserting aqi at 2018-10-02T07:00:00
Error: unable to get historical data for 2018-10-02T07:50:05
Inserting aqi at 2018-10-02T06:00:00
Inserting aqi at 2

/usr/local/lib/python3.6/dist-packages/sqlalchemy/engine/default.py:509: Warning: (1062, "Duplicate entry '2018-09-29T06:00:00' for key 'PRIMARY'")
  cursor.execute(statement, parameters)


Inserting aqi at 2018-09-29T05:00:00
Error: unable to get historical data for 2018-09-29T05:50:05
Inserting aqi at 2018-09-29T04:00:00


/usr/local/lib/python3.6/dist-packages/sqlalchemy/engine/default.py:509: Warning: (1062, "Duplicate entry '2018-09-29T04:00:00' for key 'PRIMARY'")
  cursor.execute(statement, parameters)


...

Inserting aqi at 2018-09-20T04:00:00
Inserting aqi at 2018-09-20T03:00:00
Error: unable to get historical data for 2018-09-20T03:50:05
Inserting aqi at 2018-09-20T02:00:00
Error: unable to get historical data for 2018-09-20T02:50:05
Inserting aqi at 2018-09-20T01:00:00
Error: unable to get historical data for 2018-09-20T01:50:05
Inserting aqi at 2018-09-20T00:00:00
Inserting aqi at 2018-09-19T23:00:00


/usr/local/lib/python3.6/dist-packages/sqlalchemy/engine/default.py:509: Warning: (1062, "Duplicate entry '2018-09-19T23:00:00' for key 'PRIMARY'")
  cursor.execute(statement, parameters)


Inserting aqi at 2018-09-19T22:00:00


/usr/local/lib/python3.6/dist-packages/sqlalchemy/engine/default.py:509: Warning: (1062, "Duplicate entry '2018-09-19T22:00:00' for key 'PRIMARY'")
  cursor.execute(statement, parameters)


Inserting aqi at 2018-09-19T21:00:00
Error: unable to get historical data for 2018-09-19T21:50:05


In [13]:
#schedule crontab to run on an hourly basis, get real-time data and store in database
main_real_time_aqi()

/usr/local/lib/python3.6/dist-packages/sqlalchemy/engine/default.py:509: Warning: (1007, "Can't create database 'aqi'; database exists")
  cursor.execute(statement, parameters)
/usr/local/lib/python3.6/dist-packages/sqlalchemy/engine/default.py:509: Warning: (1050, "Table 'NY_aqi' already exists")
  cursor.execute(statement, parameters)


Inserting aqi at 2018-10-02T23:00:00


/usr/local/lib/python3.6/dist-packages/sqlalchemy/engine/default.py:509: Warning: (1062, "Duplicate entry '2018-10-02T23:00:00' for key 'PRIMARY'")
  cursor.execute(statement, parameters)


### Part 4: Establish a cron process, that runs your script repeatedly over time, and stores the data in your database
Please see screenshots for output